In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout
from tensorflow.keras import Model

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


# Importing data

In [2]:
train_data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
X_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [3]:
y = train_data["label"]
X = train_data.drop("label", axis = 1)

# Preprocessing

The data is 784 pixels (28*28) and it's in grayscale so the input to the model is going to be 28 * 28 * 1 as it only has one channel.
It also helps for the data to be between 0 and 1 instead of 0 and 255 as it helps the model converge faster and wouldn't have massive gradients at initialization.

In [4]:
# Converting dataframe to tensorflow tensor then scaling the data to [0,1]
X = tf.convert_to_tensor(X, tf.float64) / 255.0
X_test = tf.convert_to_tensor(X_test, tf.float64) / 255.0
y = tf.convert_to_tensor(y)
X = tf.reshape(X, (-1, 28, 28, 1))
X_test = tf.reshape(X_test, (-1,28,28,1))

In [5]:
n1 = int(X.shape[0]*.9)
X_train, y_train = X[:n1], y[:n1]
X_valid, y_valid = X[n1:], y[n1:]

# Model training

A basic CNN with 2 conv layers and 2 hidden layers, I use dropout on every single layer with p = 0.3 except for the input where I have found that it hinders performance. 

There is some data augmentation as well with a random rotation below 10° and a random translation between 0 and .1 in both dimensions.


In [6]:
kernel_size  = 3
dropout_rate = .3

In [7]:
class MNIST(Model):
    def __init__(self):
        super().__init__()
        self.training = True
        
        # Data augmentation
        self.translate = tf.keras.layers.RandomTranslation(.1, .1)
        self.rotate = tf.keras.layers.RandomRotation(.03)
        
        self.conv1 = Conv2D(64, kernel_size, activation = "relu")
        self.dropout1 = Dropout(dropout_rate)
        
        self.conv2 = Conv2D(30, kernel_size, activation = "relu")
        self.flatten = Flatten()
        self.dropout2 = Dropout(dropout_rate)
        
        self.ln1 = Dense(1024, activation = "relu")
        self.dropout3 = Dropout(dropout_rate)
        
        self.ln2 = Dense(2048, activation = "relu")
        self.dropout4 = Dropout(dropout_rate)
        
        self.out = Dense(10)
        
    def call(self, x):
            
        x = self.translate(x, training = self.training)
        x = self.rotate(x, training = self.training)
        
        x = self.conv1(x)
        x = self.dropout1(x, training = self.training)
        
        x = self.conv2(x)
        x = self.dropout2(x, training = self.training)
        x = self.flatten(x)
        
        x = self.ln1(x)
        x = self.dropout3(x, training = self.training)
        
        x = self.ln2(x)
        x = self.dropout4(x, training = self.training)
        
        out = self.out(x)
        
        return out

In [8]:
model= MNIST()
model.build((None, 28, 28, 1))
model.summary()

Model: "mnist"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_translation (RandomT  multiple                 0         
 ranslation)                                                     
                                                                 
 random_rotation (RandomRota  multiple                 0         
 tion)                                                           
                                                                 
 conv2d (Conv2D)             multiple                  640       
                                                                 
 dropout (Dropout)           multiple                  0         
                                                                 
 conv2d_1 (Conv2D)           multiple                  17310     
                                                                 
 flatten (Flatten)           multiple                  0     

The dropout paper claims that decaying learning rate works best for NNs with dropout so that's why I chose exponential decay. As for the actual values, several were tested and this seemed to work best.

In [9]:
lr = tf.keras.optimizers.schedules.ExponentialDecay(5e-4, 500, .9)
optimizer = tf.keras.optimizers.Adam(lr)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

I use the training and testing steps straight from the tensorflow docs with minor changes.

In [10]:
@tf.function
def train_step(X:tf.Tensor, y:tf.Tensor):
    model.training = True
    
    with tf.GradientTape() as tape:
        predictions = model(X)
        loss = loss_fn(y, predictions)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    return loss

In [11]:
@tf.function
def test_step(images, labels):
    model.training = False
    predictions = model(images)
    loss = loss_fn(labels, predictions)
    return loss

I used a fairly typical training loop.

In [12]:
batch_size = 128
EPOCHS = 60

In [13]:
data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
data = data.shuffle(buffer_size = len(X_train), reshuffle_each_iteration = True, 
                          seed = 10)

This is the final training loop (without validation).

In [14]:
"""
# Training loop
for epoch in range(1, EPOCHS+1):
    
    dataset = data.batch(batch_size, drop_remainder = False)
    loss = 0
    
    for X_batch, y_batch in dataset:
        loss += train_step(X_batch, y_batch) * len(X_batch)
        
    loss /= len(X_train)

    val_loss = test_step(X_valid, y_valid)
    val_accuracy = (np.argmax(model(X_valid), 1) == np.array(y_valid)).mean()  
    
    print(f"Epoch {epoch}/{EPOCHS}: loss: {loss}, val_loss: {val_loss}, val_accuracy: {val_accuracy}")
"""

'\n# Training loop\nfor epoch in range(1, EPOCHS+1):\n    \n    dataset = data.batch(batch_size, drop_remainder = False)\n    loss = 0\n    \n    for X_batch, y_batch in dataset:\n        loss += train_step(X_batch, y_batch) * len(X_batch)\n        \n    loss /= len(X_train)\n\n    val_loss = test_step(X_valid, y_valid)\n    val_accuracy = (np.argmax(model(X_valid), 1) == np.array(y_valid)).mean()  \n    \n    print(f"Epoch {epoch}/{EPOCHS}: loss: {loss}, val_loss: {val_loss}, val_accuracy: {val_accuracy}")\n'

In [15]:
for epoch in range(EPOCHS):

    dataset = data.batch(batch_size, drop_remainder = False)
    loss = 0
    
    for X_batch, y_batch in dataset: 
        loss += train_step(X_batch, y_batch)*len(X_batch)
        
    loss /= len(X)
    print(f"Epoch {epoch+1}/{EPOCHS}: loss: {loss}")

2023-10-10 14:15:15.455232: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmnist/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-10-10 14:15:25.564425: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmnist/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Epoch 1/60: loss: 0.41428911685943604
Epoch 2/60: loss: 0.1351550668478012
Epoch 3/60: loss: 0.09914227575063705
Epoch 4/60: loss: 0.08334425836801529
Epoch 5/60: loss: 0.07100579142570496
Epoch 6/60: loss: 0.06276685744524002
Epoch 7/60: loss: 0.055649738758802414
Epoch 8/60: loss: 0.051215775310993195
Epoch 9/60: loss: 0.049323949962854385
Epoch 10/60: loss: 0.045397404581308365
Epoch 11/60: loss: 0.03997902572154999
Epoch 12/60: loss: 0.0399911031126976
Epoch 13/60: loss: 0.0377754345536232
Epoch 14/60: loss: 0.03718235716223717
Epoch 15/60: loss: 0.032407548278570175
Epoch 16/60: loss: 0.0313614197075367
Epoch 17/60: loss: 0.03147077560424805
Epoch 18/60: loss: 0.027498338371515274
Epoch 19/60: loss: 0.028238864615559578
Epoch 20/60: loss: 0.02635655365884304
Epoch 21/60: loss: 0.025728655979037285
Epoch 22/60: loss: 0.024710210040211678
Epoch 23/60: loss: 0.023347340524196625
Epoch 24/60: loss: 0.02158503420650959
Epoch 25/60: loss: 0.022218290716409683
Epoch 26/60: loss: 0.022256

Putting the model in inference mode and passing the test data in batches to support GPU's memory limitations and then creating a submission that complies to the kaggle competition format.

In [16]:
model.training = False
y_pred = []

for i in range(0, len(X_test)-100+1, 100):
    partial = np.argmax(model(X_test[i:i+100]), axis = 1)
    y_pred.append(partial)
    
y_pred = np.array(y_pred).reshape(-1)

In [17]:
image_id = np.arange(start = 1, stop = y_pred.shape[0]+1)

submission = pd.DataFrame(zip(image_id,y_pred), columns = ["ImageId","label"])
submission.set_index("ImageId",inplace = True)
submission.to_csv("submission.csv")